In [1]:
from langchain import hub
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import UnstructuredMarkdownLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI, OpenAIEmbeddings

from openai import OpenAI
from langchain.prompts import ChatPromptTemplate

from llama_parse import LlamaParse
import nest_asyncio

import pandas as pd
import os
from dotenv import load_dotenv
import csv
import time

load_dotenv()

True

In [2]:
# parse the document, save as a md file.
nest_asyncio.apply()


parser = LlamaParse(
    #api_key=,  # can also be set in your env as LLAMA_CLOUD_API_KEY
    result_type="markdown",  # "markdown" and "text" are available
    num_workers=4,  # if multiple files passed, split in `num_workers` API calls
    verbose=True,
    language="en",  # Optionally you can define a language, default=en
)

# sync
documents = parser.load_data("Orientation Documents-Spring 2024 Orientation Document.pdf")
documents

# save the parsed data
parse_path = "parsed_data.md"
with open(parse_path, "w") as f:
    f.write(documents[0].text)

Started parsing the file under job_id cac11eca-a0f1-413f-bf0d-4607d4f7c8db


In [3]:
# load the parsed data
document_path = "parsed_data.md"
loader = UnstructuredMarkdownLoader(document_path)
loaded_documents = loader.load()

In [4]:
# Split the parse data into chunks, the chunk size is 512 characters
# This parameter is very important, it will affect the performance of the model
text_splitter = RecursiveCharacterTextSplitter(chunk_size=512, chunk_overlap=20)
splits = text_splitter.split_documents(loaded_documents)

# Embed
vectorstore = Chroma.from_documents(documents=splits, 
                                    embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [5]:
# Set up prompt

template = """
Context: {context}

Question: {question}
"""

local_prompt = ChatPromptTemplate.from_template(template)

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


In [13]:
#### RETRIEVAL and GENERATION using chatGPT 3.5 

# Define the function to interact with the local LLM server
def chatGPT_llm(prompt_text):
    # Point to the local server
    client = OpenAI()

    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[
        {"role": "system", "content": "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. Don't state your role and task. Avoid using prefaces"},
        {"role": "user", "content": str(prompt_text)}
    ],
    temperature=0.8,
    )
    return str(completion.choices[0].message.content)

# Update the chain to use the local LLM server
rag_chain_ChatGPT = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | local_prompt
    | chatGPT_llm
    | StrOutputParser()
)

# Question
rag_chain_ChatGPT.invoke("What is this program?")

'Can you please provide more context or clarify your question regarding the program mentioned in the message?'

In [16]:
#### RETRIEVAL and GENERATION using Llama CPP###
# The Llama model is hosted locally and can be accessed via the OpenAI API

# Define the function to interact with the local LLM server
def local_llm(prompt_text):
    # Point to the local server
    client = OpenAI(base_url="http://192.168.68.78:8080/v1")

    completion = client.chat.completions.create(
    model="Meta-Llama-3-8B-Instruct-GGUF",
    messages=[
        {"role": "system", "content": "You are an assistant for question-answering tasks. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Use three sentences maximum and keep the answer concise. Don't state your role and task. Avoid using preface like According to the provided context."},
        {"role": "user", "content": str(prompt_text)}
    ],
    temperature=0.8,
    )
    return str(completion.choices[0].message.content)

# Update the chain to use the local LLM server
rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | local_prompt
    | local_llm
    | StrOutputParser()
)
llama_response = rag_chain.invoke("What is this program?")
llama_response[:-10]

'This program is a concentration or specialization program within a program of study. It requires students to be comfortable working with multiple programming languages and have taken advanced topics in computer science.'

In [9]:
# load evaluation questions
questions = pd.read_csv("faq_output.csv")
questions

,category,qns_no,qns,ans
0,New Student Onboarding/GT Email Account,1,Where do I send my official transcript(s) and ...,Send your final official transcript(s) and oth...
1,New Student Onboarding/GT Email Account,2,What happens if I do not submit my official tr...,"If you are a new student starting Spring 2024,..."
2,New Student Onboarding/GT Email Account,3,Is there an orientation for the OMSCS program?,"Since this is an online program, we do not hav..."
3,New Student Onboarding/GT Email Account,4,What are the expectations regarding my GT emai...,"The Institute, as well as the department, will..."
4,New Student Onboarding/GT Email Account,5,How can I access my GT email account?,Please visit this website for information on h...
...,...,...,...,...
153,Graduation,13,Is it possible to graduate with honors/distinc...,Graduating with distinction/honors is not avai...
154,Graduation,14,Can I continue taking OMSCS courses after I gr...,It is possible for students to continue taking...
155,Graduation,15,Can I continue being a TA after I graduate fro...,"This is possible, but it is more complicated t..."
156,Graduation,16,What technology-related services will I retain...,Please visit https://gatech.service-now.com/ho...


In [10]:
# This is a random question generator
def get_sample_questions():
    random_question = questions.sample().iloc[0]
    return {'question':random_question.qns, 'answer':random_question.ans}

print(get_sample_questions())

{'question': 'Can OMSCS Advising tell me which courses are open currently?', 'answer': 'Students can access up-to-date enrollment and wait list totals by performing a class search in OSCAR. You can\nperform a course look-up in OSCAR once you have logged in using your GT credentials — the full courses have a\n“C” in the first column, and the open courses have a checkbox that can be selected in order to enroll.'}


In [11]:
# Get sample questions and invoke the chains once

start_time = time.time()
q_and_a = get_sample_questions()
print('The question is:', q_and_a.get('question'))
print()

# Invoke chatGPT
start_time = time.time()
print('chatGPT answer:', rag_chain_ChatGPT.invoke(q_and_a.get('question')))
print('Time to invoke chatGPT:', round(time.time() - start_time,2),'seconds')
print()

# Invoke local LLAMA
start_time = time.time()
print('Local LLAMA answer:', rag_chain.invoke(q_and_a.get('question')))
print('Time to invoke local LLAMA:', round(time.time() - start_time,2),'seconds')
print()

print('The official answer is:', q_and_a.get('answer'))

The question is: What are the rules regarding the Satisfactory Academic Progress policy?

chatGPT answer: The rules regarding the Satisfactory Academic Progress policy state that students must maintain satisfactory academic progress to continue in the OMSCS program. Continuous withdrawal from semesters may lead to academic dismissal, with warnings issued in advance. While there is no specific limit on withdrawing from or repeating a course, students should adhere to the degree requirements, including those related to satisfactory academic progress, the six-year requirement, and grade substitution policies.
Time to invoke chatGPT: 1.54 seconds

Local LLAMA answer: Students must make satisfactory academic progress to continue enrolling in the OMSCS program, and failing to do so may lead to academic dismissal. According to the FAQ, there is no specific limit to the number of times a student can take and withdraw from a course, but students should consider the degree requirements and satis

In [17]:
count = 0

with open('faq_output_with_answers_compare_new.csv', 'w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['Question', 'Answer', 'chatGPT_answer', 'Local_LLAMA_answer','chat_gpt_time','local_llama_time'])

    for row in questions.iterrows():
        question = row[1].qns
        answer = row[1].ans
        start_time = time.time()
        chatGPT_answer = rag_chain_ChatGPT.invoke(question)
        chat_gpt_time = round(time.time() - start_time,2)

        start_time = time.time()
        Local_LLAMA_answer = rag_chain.invoke(question)
        Local_LLAMA_answer = Local_LLAMA_answer[:-10]
        local_llama_time = round(time.time() - start_time,2)

        print(f'Finished generating answer for question {count}.')
        count+=1

        # write the row to the CSV file
        writer.writerow([question, answer, chatGPT_answer, Local_LLAMA_answer, chat_gpt_time, local_llama_time])

Finished generating answer for question 0.
Finished generating answer for question 1.
Finished generating answer for question 2.
Finished generating answer for question 3.
Finished generating answer for question 4.
Finished generating answer for question 5.
Finished generating answer for question 6.
Finished generating answer for question 7.
Finished generating answer for question 8.
Finished generating answer for question 9.
Finished generating answer for question 10.
Finished generating answer for question 11.
Finished generating answer for question 12.
Finished generating answer for question 13.
Finished generating answer for question 14.
Finished generating answer for question 15.
Finished generating answer for question 16.
Finished generating answer for question 17.
Finished generating answer for question 18.
Finished generating answer for question 19.
Finished generating answer for question 20.
Finished generating answer for question 21.
Finished generating answer for question 22

KeyboardInterrupt: 